# Model Training with Scikit-learn



Train a model that predicts whether or not a patient has diabetes, based on medical features. 



### 1. Import the required libraries and packages.

In [1]:
from typing import List, Dict
from pymongo import MongoClient



import pandas as pd



from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

### 2. Connect to MongoDB.

In [3]:
client = MongoClient('mongodb+srv://xxxx')  # Adjust the URI as needed
db = client['patients']  # Replace with your database name
collection = db['diabetes']  # Replace with your collection name

### 3. Load and Preprocess the data.



Split the data into two data frames: features (`X`) and target variable (`y`).

In [5]:
# Step 2: Load features (X) and target (y) from MongoDB
cursor_features = collection.find({}, {'_id': 0, 'Outcome': 0})  # Exclude 'Outcome' and '_id'
X = pd.DataFrame(list(cursor_features))

cursor_target = collection.find({}, {'_id': 0, 'Outcome': 1})  # Include only 'Outcome'
y = pd.DataFrame(list(cursor_target))['Outcome']

Inspect the two dataframes.

In [6]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [7]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

Divide the data into training and test data sets. 



The `train_test_split` method of Scikit-learn can split the data set into random train and test subsets.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(

    X,

    y,

    test_size=0.2,

    random_state=0

)



print(f"Number of samples in training set: {X_train.shape[0]}")

print(f"Number of samples in test set: {X_test.shape[0]}")

Number of samples in training set: 614
Number of samples in test set: 154


### 4. Create and train the model.

Create an instance of one of the models available in Scikit-learn and train the model with the training set. For this particular example, you can use a logistic regression model, which is a simple approach to solve classification problems.

In [9]:
# Instantiate the model with hyper parameters

model = LogisticRegression(penalty="l2", C=1.0, max_iter=300)



# Train the model

model.fit(X_train, y_train)

LogisticRegression(max_iter=300)

### 5. Evaluate the model metrics.



After the model is trained, evaluate the model against the test set.

In [10]:
# Compute the model predictions for the test data: y_predicted

y_predicted = model.predict(X_test)



# Compare the predicted values for the test set (y_predicted)

# against the expected values (y_test)

print("Classification report:")

print(classification_report(y_test, y_predicted))

Classification report:
              precision    recall  f1-score   support

           0       0.84      0.92      0.88       107
           1       0.76      0.62      0.68        47

    accuracy                           0.82       154
   macro avg       0.80      0.77      0.78       154
weighted avg       0.82      0.82      0.82       154



The trained model has an accuracy value of nearly 82%.



You can improve the score by retraining the model after more sophisticated data engineering or by tweaking the model hyper parameters.

### 6. Test the model with sample cases.

Test the model with data from two patients: one patient with diabetes and one patient without diabetes.

In [12]:
# Tuple for textual display of prediction

classes = ('No diabetes', 'Diabetes')





def predict(patients: List[Dict]):

    inputs = pd.DataFrame(patients)

    predictions = model.predict(inputs)

    return [classes[p] for p in predictions]





diabetes_patient = {

    "Pregnancies": 6.0,

    "Glucose": 110.0,

    "BloodPressure": 65.0,

    "SkinThickness": 15.0,

    "Insulin": 1.0,

    "BMI": 45.7,

    "DiabetesPedigreeFunction": 0.627,

    "Age": 50

}



no_diabetes_patient = {

    "Pregnancies": 0,

    "Glucose": 88.0,

    "BloodPressure": 60.0,

    "SkinThickness": 35.0,

    "Insulin": 1.0,

    "BMI": 45.7,

    "DiabetesPedigreeFunction": 0.27,

    "Age": 20

}





predictions = predict([diabetes_patient, no_diabetes_patient])

print(predictions)

['Diabetes', 'No diabetes']
